In [3]:
import pandas as pd
from time import sleep 
from random import randint
from selenium import webdriver
driver = webdriver.Chrome('H:\Digital_Media\climate_change\climate-chatbot\chromedriver_win32\chromedriver.exe') 

In [4]:
# Get data from url by class
for url in open("list_of_urls.txt").readlines():
    driver.get(url)
    items = len(driver.find_elements_by_class_name("quote"))
    total = []
    print(url)
    for item in range(items):
        quotes = driver.find_elements_by_class_name("quote")
        for quote in quotes:
            quote_text = quote.find_element_by_class_name('text').text[1:-2]
            author = quote.find_element_by_class_name('author').text
            new = ((quote_text,author))
            total.append(new)
    df = pd.DataFrame(total,columns=['quote','author'])
    df.to_csv('quoted.csv')
    sleep_for = randint(300,2500) / 1000.0
    sleep( sleep_for )
driver.close()

http://quotes.toscrape.com/js/page/1/


In [ ]:
for url in open("list_of_urls.txt").readlines():
    driver.get(url)
    
    sleep_for = randint(300,2500) / 1000.0
    sleep( sleep_for )
driver.close()

In [12]:
import json
myJson = {
    "question": "first question",
    "author": 
    {
        "auto": "auto1",
        "name": "nazmul hossain"
    }
}
s = json.dumps(myJson)
with open("myJson.json", "w") as f:
    f.write(s)